## nDCG

In [ ]:
test_df = pd.DataFrame({'total_points': [3, 2, 3, 0, 1, 2], 'llm-heapsort_rank': [1, 2, 3, 4, 5, 6]})
test_df

,total_points,llm-heapsort_rank
0,3,1
1,2,2
2,3,3
3,0,4
4,1,5
5,2,6


In [ ]:
def nDCG(df: pd.DataFrame):
    """
    Computes the normalized discounted cumulative gain (nDCG)
    for a DataFrame that contains the columns 'total_points' (ground truth relevance)
    and 'llm-heapsort_rank' (predicted score).
    
    Parameters:
        df (pd.DataFrame): DataFrame with 'total_points' and 'llm-heapsort_rank' columns.
        
    Returns:
        float: The nDCG value.
    """
    # --- Compute DCG using the predicted order ---
    # Sort by predicted scores in descending order.
    df_pred = df.sort_values('llm-heapsort_rank', ascending=True).reset_index(drop=True)
    # Create a numpy array of positions (0-indexed) and compute the discount factors.
    positions = np.arange(len(df_pred))
    discounts = np.log2(positions + 2)  # since log2(1+position) with positions starting at 1
    
    gains = df_pred['total_points']
    dcg = np.sum(gains / discounts)
    
    # --- Compute Ideal DCG (IDCG) using the ideal (true relevance) order ---
    df_ideal = df.sort_values('total_points', ascending=True).reset_index(drop=True)
    positions = np.arange(len(df_ideal))
    discounts = np.log2(positions + 2)
    gains = 2 ** df_ideal['total_points'] - 1
    idcg = np.sum(gains / discounts)
    
    # Avoid division by zero if idcg is 0.
    if idcg == 0:
        return 0.0
    
    print(f"DCG: {dcg}\nIDCG: {idcg}")
    return dcg / idcg

print(f"nDCG: {nDCG(df)}")

DCG: 102.9770403454842
IDCG: 2450393.6716409237
nDCG: 4.202469241463756e-05


In [ ]:
def normalized_dcg(df, relevance_col='relevance'):
    """
    Computes the Normalized Discounted Cumulative Gain (NDCG) for a given ranking.
    
    The DataFrame is assumed to have one column containing the relevance scores 
    (by default, this column is named 'relevance'). The order of the rows in the DataFrame 
    should reflect the ranking order you wish to evaluate.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing at least the relevance scores.
        relevance_col (str): The name of the column with the relevance scores.
                             Defaults to 'relevance'.
    
    Returns:
        float: The normalized discounted cumulative gain.
    """
    def dcg(scores):
        # Convert scores to a NumPy array
        scores = np.asarray(scores)
        # Create an array of discount factors: log2(1 + rank) where rank starts at 1.
        discounts = np.log2(np.arange(2, scores.size + 2))
        # Compute DCG using the formula: sum((2^score - 1) / discount)
        return np.sum((2 ** scores - 1) / discounts)
    
    # Compute the DCG for the current ranking (order of the DataFrame)
    actual_dcg = dcg(df[relevance_col])
    
    # Compute the ideal DCG (IDCG) by sorting the relevance scores in descending order
    ideal_dcg = dcg(df[relevance_col].sort_values(ascending=False))
    
    # Return normalized DCG; avoid division by zero.
    print(f"{actual_dcg=}, {ideal_dcg=}")
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0.0

normalized_dcg(test_df, 'total_points')

actual_dcg=np.float64(6.861126688593502), ideal_dcg=np.float64(7.1409951840957)


np.float64(0.9608081943360617)